In [1]:
%load_ext sql
%sql --read-only

In [13]:
%%sql
select 
    sum(PieceSize / 1024 ** 5) as total_stored_data 
from filecoin_state_market_deals 
where 
    SectorStartEpoch != -1 and SlashEpoch != -1 limit 10

Running query in 'duckdb:////workspaces/filecoin-analytics/data/dbt.duckdb'

total_stored_data
148


In [14]:
%%sql
select
    *
from filecoin_state_market_deals
limit 10

Running query in 'duckdb:////workspaces/filecoin-analytics/data/dbt.duckdb'

DealID,PieceCID,PieceSize,VerifiedDeal,Client,Provider,Label,StartEpoch,EndEpoch,StoragePricePerEpoch,ProviderCollateral,ClientCollateral,SectorStartEpoch,LastUpdatedEpoch,SlashEpoch,VerifiedClaim
159155,{'/': 'baga6ea4seaqaazz4brpn3zde6dkorq3muck42qzw5hetx7wtwm6oyz4lqdklcpa'},65536,False,f020319,f016305,mAXESINiz1A6zzmVo8QoU7wKbP+gB3PGSyA43a4ZukzHIxc3z,1281643200,1282165340,0,0,0,-1,-1,-1,0
159170,{'/': 'baga6ea4seaqaazz4brpn3zde6dkorq3muck42qzw5hetx7wtwm6oyz4lqdklcpa'},65536,False,f020319,f016305,mAXESINiz1A6zzmVo8QoU7wKbP+gB3PGSyA43a4ZukzHIxc3z,1282046400,1282568540,0,0,0,-1,-1,-1,0
159181,{'/': 'baga6ea4seaqaazz4brpn3zde6dkorq3muck42qzw5hetx7wtwm6oyz4lqdklcpa'},65536,False,f020319,f016309,mAXESINiz1A6zzmVo8QoU7wKbP+gB3PGSyA43a4ZukzHIxc3z,1282608000,1283129618,0,0,0,-1,-1,-1,0
159183,{'/': 'baga6ea4seaqcjw5o3bbrn6u2tgv7j7lihxufsm4c3zijhznymvpjfmlckbuccaq'},1048576,False,f020319,f016305,mAXESIN6mq1OV8vx3Wq/vL7IaZX5f+bmEIatiFHnvR4ZNhLVr,1282608000,1283130140,0,0,0,-1,-1,-1,0
159188,{'/': 'baga6ea4seaqkkyehwbcfh6l72e2jrooyqvbbr6f2xdyobesu6aupjfrox7tc2ki'},4096,False,f020319,f016304,mAXESIOBDGaYV/UAiPSsJZDXHqoU4SYqq7nJ0/tagst+ls8d9,1282680000,1283202204,0,0,0,-1,-1,-1,0
159189,{'/': 'baga6ea4seaqojxopcje7y4biwj3qs2xt3srmcill6uvmga6gfqcppcutqs3g6my'},4096,False,f020319,f016305,mAXESIAjeNqjrNMiI9wdUEG54r2AaDGf1WfvfXjuutGQ/esch,1282680000,1283202140,0,0,0,-1,-1,-1,0
159190,{'/': 'baga6ea4seaqpg2hxv5os6bygcipqks537txiaojypb6ilzsu5qmp7bw2dptwcei'},4096,False,f020319,f016303,mAXESIKtGtBrFsswfb5KZRnLb06BxpxE2ZJiuTQjWwUEy4dT5,1282680000,1283204137,0,0,0,-1,-1,-1,0
159191,{'/': 'baga6ea4seaqoxyynqy3cbeutvbiygtlk6jw6wc4cpolguq65ebgrby4tcyjbgci'},4096,False,f020319,f016309,mAXESIFcZ1jlnoL7/DPtkuvcTlV0aYMyeLo5YL5ivYcTLf+lj,1282680000,1283201618,0,0,0,-1,-1,-1,0
159192,{'/': 'baga6ea4seaqgxbzhjoaxjpzcqozmb6nznm6y452okpmp6adqmn6p4bz6w5ugaoq'},4096,False,f020319,f016303,mAXESIFvyl9nFfaMUEQ2XoY8SDTnhpybAFU/hcn98nkyQnVl6,1282680000,1283204137,0,0,0,-1,-1,-1,0
159193,{'/': 'baga6ea4seaqkwr7zajmbufki6sgx2f3cqcjbq3bh4is4u3j6tlv2crfyyiyjykq'},8192,False,f020319,f016309,mAXESIHyfVbyMVkTSYC2mCSewpvokw5I8LEBwQuwEvTsvAios,1282694400,1283216018,0,0,0,-1,-1,-1,0
